In [16]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate

In [5]:
# load pdf
pdfLoader = PyMuPDFLoader(file_path="ADD-953.pdf")
pdfPages = pdfLoader.load()
# consolidate all text contents in a single string
content_list = [page.page_content for page in pdfPages]
consolidatedTexts = '\n\n'.join(content_list)


In [7]:
# split consolidated texts string into chunks
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_text(consolidatedTexts)


Created a chunk of size 533, which is longer than the specified 500
Created a chunk of size 502, which is longer than the specified 500
Created a chunk of size 597, which is longer than the specified 500
Created a chunk of size 592, which is longer than the specified 500
Created a chunk of size 639, which is longer than the specified 500
Created a chunk of size 606, which is longer than the specified 500
Created a chunk of size 584, which is longer than the specified 500


In [ ]:
# Convert chunks into embeedings
embeder = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-V2")
embeddings = embeder.embed_documents(chunks)


/var/folders/2w/cqz3z_yn04n_r99f8b5w1frc0000gp/T/ipykernel_2114/687850255.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeder = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-V2")


In [ ]:
# Store embedding in a vector store
vectorstore = FAISS.from_texts(texts=content_list, embedding=embeder)
# Get retriever from vector store to retrieve related contents for user queries
retriever = vectorstore.as_retriever()

In [18]:

# Create a prompt template with variables (context and user query)
system_prompt_template = "Provide helpful answers based on the following context: {context} in a neated bullet points and also provide a mindmap for the entire context."
chat_prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_prompt_template),
    HumanMessagePromptTemplate.from_template("User query: {query}")
])


In [ ]:
# Create an llm
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import os
load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [24]:
# Chain input Params -> prompt template -> llm -> output parser
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
chain = (
    {"context": retriever, "query": RunnablePassthrough()}
    | chat_prompt_template
    | llm
    | StrOutputParser()
)


In [28]:
def corrective_rag(user_query):
    first_answer = llm.invoke(user_query)
    context = chain.invoke(user_query)
    corretive_rag_prompt_template = f"""
You previously answered the question:

Answer: {first_answer}

But here is additional retrieved context that may change or improve the response:

Context: {context}

Rewrite your answer using the retrieved context to ensure it is accurate and grounded in evidence.
If the retrieved context contradicts your initial answer, correct it.
"""
    return llm.invoke(corretive_rag_prompt_template)
result = corrective_rag("Why was the TVE authorization model chosen over the existing MVPD partner subscription flow, and what contractual obligations drove this choice?")
# Parse llm output and display it
from IPython.display import display, Markdown
display(Markdown(result.content))

The decision to use the TVE (TV Everywhere) authorization model for TNT Sports MVP, rather than integrating it with WBD's existing "partner subscription model" (which handles DTC/IAP/MVPD subscriptions on its Bolt platform), was driven by a specific set of operational, time-sensitive, and fundamental conflicts, alongside the inherent contractual obligations of the TVE ecosystem.

Here's a breakdown based on the provided context:

### Specific Reasons for Choosing TVE Authorization Model for TNT Sports MVP:

1.  **Complexity Avoidance:**
    *   Integrating TVE authorizations with WBD's existing DTC (Direct-to-Consumer), IAP (In-App Purchase), and MVPD (Multichannel Video Programming Distributor) subscription flows would introduce extreme complexity.
    *   Scenarios like managing hybrid DTC billing for content a TVE user lacks access to (e.g., a TVE user only has TNT, but wants TruTV, which might be bundled with a DTC offering) or determining asset-level authorizations when a user has both a DTC/IAP subscription *and* a TVE link, presented significant integration challenges.
    *   To simplify, a mutually exclusive model (either pure TVE or WBD-managed subscription) was mandated.

2.  **Limited Time to Launch:**
    *   Given the tight launch schedule for TNT Sports MVP, the need to avoid the aforementioned complexities led to the decision for a mutually exclusive model, prioritizing speed and simplicity of implementation.

3.  **Conflict with TVE Basic Principles:**
    *   WBD's existing "Bolt partner subscription model" (used for services like HBO Max and Discovery+) is designed for WBD to create and manage customer accounts. This model requires WBD to collect minimal registration details (like email ID or phone number) to establish a direct relationship with the customer for marketing, support, and extended features like global roaming.
    *   However, this "partner subscription model" is explicitly stated as being **"counter to TVE basic principles."** In the traditional TVE model, the pay television provider (MVPD) acts as the **identity provider**, managing customer identity and entitlements. WBD, as the **service provider**, is generally **not allowed to create native accounts or collect personally identifiable information (PII)** from the customer, unless a specific exception is granted. This fundamental conflict made it impractical to use WBD's existing account-centric subscription flow for a pure TVE offering.

### Contractual Obligations Driving This Choice (Inherent to TVE):

The decision was also heavily influenced by the inherent roles and restrictions defined within the TV Everywhere (TVE) business model's carriage agreements:

1.  **Identity Provider Role of MVPDs:**
    *   In the TVE framework, the MVPD is contractually designated as the identity provider. They are responsible for authenticating the subscriber and verifying their entitlements (i.e., which channels they subscribe to). This means the MVPD owns the customer's identity verification process for TVE access.

2.  **Service Provider Role and Restrictions on WBD:**
    *   WBD, as the service provider, is solely responsible for fulfilling content access once the user is authenticated by the MVPD. Crucially, and as mentioned above, WBD is generally **contractually prohibited from creating native accounts or collecting PII** from the customer when operating under a pure TVE model, unless an explicit exception is part of the agreement. This directly clashes with WBD's standard "partner subscription model" which requires account creation and PII collection.

3.  **Brand Attribution:**
    *   As a service provider, WBD may also be contractually obligated to credit content access to the identity provider (the MVPD) through explicit co-branding within the streaming experience. This reinforces the MVPD's role in providing the core subscription.

### Broader Context (from Original Answer, still relevant to TVE's existence):

While the immediate reasons for TNT Sports MVP's choice were operational and platform-specific, the overarching strategic reasons for the *existence* of the TVE model remain:

*   **Protecting Existing Revenue Streams for Content Providers:** TVE was a compromise to protect billions in carriage fees paid by MVPDs to content providers. By requiring an existing MVPD subscription, TVE prevented content providers from cannibalizing MVPD bundles by offering their linear channels directly.
*   **Maintaining MVPD Relationships:** Content providers rely on MVPDs for mass audience reach. TVE helped maintain these crucial partnerships by presenting online access as an added value to the MVPD subscription, rather than a direct threat.
*   **Cost Avoidance (for MVPD integration):** While WBD has its own platform, the general TVE model allows content providers to leverage MVPDs' existing authentication infrastructure, reducing the need to build entirely new billing and customer management systems for *pure TVE* access.

In summary, for TNT Sports MVP, the choice of a mutually exclusive TVE authorization model was a pragmatic decision to navigate the complexities of integrating with WBD's existing subscription platform, meet tight deadlines, and, most importantly, adhere to the fundamental architectural and contractual principles of TVE, which explicitly limit WBD's ability to manage customer accounts and collect PII for TVE users. This stood in direct opposition to WBD's established "partner subscription model."